In [ ]:
from evaluate_task import Task
from evaluate_task import evaluate_multiple_choice_task 
from evaluate_task import evaluate_free_response_task

GPT3_5 = "gpt-3.5-turbo-1106"
GPT4 = "gpt-4-0125-preview"

anachronism = Task('multiple_choice',
                   'data/task_anachronisms.json',
                   'Does the following sentence contain non-contemporaneous (anachronistic) elements?.')
logical_sequence = Task('multiple_choice',
                        'data/task_logical_sequence.json',
                        "Identify the correct chronological or sequential order of items in a list.")
color_HCL = Task('multiple_choice', 'data/task_color_HCL.json',"")
color_RGB = Task('multiple_choice', 'data/task_color_RGB.json',"")
color_HEX = Task('multiple_choice', 'data/task_color_HEX.json',"")
color_HSL = Task('multiple_choice', 'data/task_color_HSL.json',"")
matrixshapes = Task('free_response',
                    'data/task_matrixshapes.json',
                    "Keep track of matrix shapes through various transformations.")
operators = Task('free_response',
                    'data/task_operators.json',
                    "Given the definition of the op operator, compute the result.")


In [ ]:
anachronism.evaluate(GPT3_5)

In [ ]:
import wandb
# start a new wandb run to track this script
wandb.init(project="LLMs-Still-Can-t",config={
    "model": GPT4,
    "dataset": anachronism.json_string,
    "shot": 0})

# evaluate the task n times
for i in range(1):
    acc , df = anachronism.evaluate(wandb.config.model)
    wandb.log({"System Prompt": anachronism.system_prompt, 
                "Name":f"Baseline [{anachronism.json_string}]", 
                "accuracy": acc, 
                "examples": wandb.Table(dataframe=df)})
wandb.finish()